In [1]:
import pandas as pd
import pandas_checks as pdc

import src

FILE_IN="data/interim/gnomad_snrna_variants_tidy.tsv"

In [10]:
df = (
    pd.read_csv(FILE_IN, sep="\t", na_values=".")
    .pipe(
        lambda x: x.assign(
            **pd.DataFrame(
                x["bed_info"].str.split(",").to_list(),
                columns=["ensg", "symbol", "length"],
                index=x.index,
            )
        ).drop(columns=["bed_info"])
    )
    .astype({"length": "int"})
    .check.head()
)

""" We want:
- Gene symbols
- Variant types (i.e. indel vs SNV)
- Unique variants
- snRNA vs Pseudogene
- HWE statistics (autosomes only)
-- Strongest hits
-- Genes with the greatest number of genome-wide significant HWE outliers
"""

<h5 style='text-align: left'><span style='color:None; background-color:None'>First 5 rows</span></h5>

,chrom,pos,ref,alt,ac,an,af,nhomalt,ac_nfe,an_nfe,af_nfe,nhomalt_nfe,af_popmax,allele_type,cadd_phred,ensg,symbol,length
0,chr1,157840,ATCTC,A,1,29526,0.000,0,1,13154,0.000,0,0.000,del,1.546,ENSG00000222623.1,RNU6-1100P,104
1,chr1,758239,A,T,2,146046,0.000,0,1,68022,0.000,0,0.000,snv,1.479,ENSG00000223181.1,RNU6-1199P,104
2,chr1,758240,T,C,2,146026,0.000,0,1,68014,0.000,0,0.000,snv,1.903,ENSG00000223181.1,RNU6-1199P,104
3,chr1,758245,T,C,4207,145902,0.029,168,2678,67982,0.039,79,0.039,snv,1.843,ENSG00000223181.1,RNU6-1199P,104
4,chr1,758254,A,G,1,146004,0.000,0,0,68002,0.000,0,0.000,snv,2.970,ENSG00000223181.1,RNU6-1199P,104


<h5 style='text-align: left'><span style='color:None; background-color:None'>Data types</span></h5>

,
chrom,object
pos,int64
ref,object
alt,object
ac,int64
an,int64
af,float64
nhomalt,int64
ac_nfe,int64
an_nfe,int64
